In [1]:
import os
import torch
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric,Dataset,concatenate_datasets,set_caching_enabled, ClassLabel
import pandas as pd

import random
from IPython.display import display, HTML

import json
from transformers import Wav2Vec2CTCTokenizer,Wav2Vec2ForCTC,Wav2Vec2Processor,Trainer,TrainingArguments,Wav2Vec2FeatureExtractor

import re
set_caching_enabled(False)

import soundfile as sf
import torchaudio


import IPython.display as ipd
import numpy as np
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from tqdm import tqdm
import torch

# Set environment variables
os.environ['WANDB_DISABLED '] = 'True'

import warnings
warnings.filterwarnings('ignore')

In [2]:
import logging
import transformers
transformers.logging.get_verbosity = lambda: logging.NOTSET

transformers.logging.get_verbosity()



0

In [3]:
import datasets
datasets.logging.get_verbosity = lambda: logging.NOTSET

In [4]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [5]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [6]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [7]:
import glob 
data_dir  = "/workspace/data_processed/*"
data_dir_list = glob.glob(data_dir)
data_dir_list.remove('/workspace/data_processed/5')

In [8]:
data_dir_list

['/workspace/data_processed/26',
 '/workspace/data_processed/13',
 '/workspace/data_processed/2',
 '/workspace/data_processed/7',
 '/workspace/data_processed/14',
 '/workspace/data_processed/24',
 '/workspace/data_processed/3',
 '/workspace/data_processed/38',
 '/workspace/data_processed/28',
 '/workspace/data_processed/16',
 '/workspace/data_processed/30',
 '/workspace/data_processed/36',
 '/workspace/data_processed/4',
 '/workspace/data_processed/18',
 '/workspace/data_processed/29',
 '/workspace/data_processed/8',
 '/workspace/data_processed/31',
 '/workspace/data_processed/27',
 '/workspace/data_processed/6',
 '/workspace/data_processed/10',
 '/workspace/data_processed/21',
 '/workspace/data_processed/15',
 '/workspace/data_processed/35',
 '/workspace/data_processed/9',
 '/workspace/data_processed/25',
 '/workspace/data_processed/22',
 '/workspace/data_processed/33',
 '/workspace/data_processed/20',
 '/workspace/data_processed/12',
 '/workspace/data_processed/39',
 '/workspace/data

In [9]:
data_set_list_total = []
for i in range(len(data_dir_list)):
    common_voice_val = Dataset.load_from_disk(data_dir_list[i])
    data_set_list_total.append(common_voice_val)


In [10]:
len(data_set_list_total)

38

In [11]:
len((data_set_list_total[:-1]))

37

In [12]:
len((data_set_list_total[-1:]))

1

In [13]:
common_voice_train = concatenate_datasets(data_set_list_total[:-1])

In [14]:
common_voice_test = concatenate_datasets(data_set_list_total[-1:])

In [15]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 323525
})

In [16]:
common_voice_test

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 10000
})

In [17]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 323525
})

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [19]:
common_voice_train

Dataset({
    features: ['input_values', 'labels'],
    num_rows: 323525
})

In [20]:
repo_name = "wav2vec2-large-xlsr-300m-nepali"

In [21]:
tokenizer.push_to_hub(repo_name)

In [22]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [23]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [24]:
wer_metric = load_metric("wer")

In [25]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [26]:
len(processor.tokenizer)

62

In [27]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.0,
    hidden_dropout=0.0,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.0,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.codevectors', 'quantizer.weight_proj.weight', 'project_q.weight', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [28]:
model.freeze_feature_extractor()

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=100,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  push_to_hub=True,
)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

/workspace/wav2vec2-large-xlsr-300m-nepali is already a clone of https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali. Make sure you pull the latest changes with `repo.git_pull()`.
Using amp half precision backend


In [31]:
trainer.train()

***** Running training *****
  Num examples = 323525
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 1011000


Step,Training Loss,Validation Loss,Wer
400,6.657500,3.322663,1.000000
800,2.041400,1.162285,0.871682
1200,1.109100,0.900162,0.734135
1600,0.975700,0.791136,0.683623
2000,0.891200,0.740895,0.642548
2400,0.841800,0.697942,0.606249
2800,0.831700,0.676051,0.591271
3200,0.796000,0.658611,0.568378
3600,0.789000,0.649247,0.562919
4000,0.768300,0.634902,0.564559


***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xlsr-300m-nepali/checkpoint-400
Configuration saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-400/config.json
Model weights saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-400/pytorch_model.bin
Configuration saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-400/preprocessor_config.json
Configuration saved in wav2vec2-large-xlsr-300m-nepali/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 10000
  Batch size = 8
Saving model checkpoint to wav2vec2-large-xlsr-300m-nepali/checkpoint-800
Configuration saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-800/config.json
Model weights saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-800/pytorch_model.bin
Configuration saved in wav2vec2-large-xlsr-300m-nepali/checkpoint-800/preprocessor_config.json
Configuration saved in wav2vec2-large-xlsr-300m-nepali/preprocessor_config.json
***** Running Evalua

KeyboardInterrupt: 

In [32]:
trainer.push_to_hub()

Saving model checkpoint to wav2vec2-large-xlsr-300m-nepali
Configuration saved in wav2vec2-large-xlsr-300m-nepali/config.json
Model weights saved in wav2vec2-large-xlsr-300m-nepali/pytorch_model.bin
Configuration saved in wav2vec2-large-xlsr-300m-nepali/preprocessor_config.json
Dropping the following result as it does not have all the necessary fields:
{}
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.37k/1.18G [00:00<?, ?B/s]

To https://huggingface.co/prajin/wav2vec2-large-xlsr-300m-nepali
   2889e67..e7edbaf  main -> main



In [35]:
# trainer.save_metrics(all, metrics)